# Compiling Senator and House of Representatives Stock Trading Data


In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [36]:
senator_trades = pd.read_csv("data/cleaned/all_transactions_senate.csv")
senator_trades = senator_trades.drop("Unnamed: 0", axis=1)
senator_trades.head()

,transaction_date,owner,ticker,type,amount,comment,senator,transaction_year,transaction_month,transaction_day,amount_lower,amount_upper
0,2021-08-18,Joint,SSYS,Purchase,"$1,001 - $15,000",--,Thomas H Tuberville,2021,8,18,1001,15000
1,2021-08-12,Joint,SSBK,Purchase,"$50,001 - $100,000",--,Thomas H Tuberville,2021,8,12,50001,100000
2,2021-08-23,Spouse,LMRK,Sale (Full),"$1,001 - $15,000",--,Thomas R Carper,2021,8,23,1001,15000
3,2021-08-11,Spouse,WW,Purchase,"$1,001 - $15,000",--,Thomas R Carper,2021,8,11,1001,15000
4,2021-08-04,Spouse,ARE,Sale (Partial),"$1,001 - $15,000",--,Thomas R Carper,2021,8,4,1001,15000


In [39]:
# senator_trades["type"].unique()
senator_trades[senator_trades["type"] == "Exchange"]

,transaction_date,owner,ticker,type,amount,comment,senator,transaction_year,transaction_month,transaction_day,amount_lower,amount_upper
102,2021-06-02,Self,MRK,Exchange,"$1,001 - $15,000",--,John W Hickenlooper,2021,6,2,1001,15000
107,2021-05-17,Self,FL4.SG,Exchange,"$15,001 - $50,000",FLIR Systems Merger With TeledyneAsset Held in...,"William F Hagerty, Iv",2021,5,17,15001,50000
114,2020-04-01,Joint,VZ,Exchange,"$1,001 - $15,000",Exchange occurred as a result of the Sprint/T-...,"Jerry Moran,",2020,4,1,1001,15000
151,2021-02-05,Spouse,DD,Exchange,"$1,001 - $15,000",--,Thomas R Carper,2021,2,5,1001,15000
223,2020-12-15,Self,AIV,Exchange,"$15,001 - $50,000",Exchange of stock due to partial corporate spi...,Patrick J Toomey,2020,12,15,15001,50000
...,...,...,...,...,...,...,...,...,...,...,...,...
4738,2016-06-01,Spouse,WCN,Exchange,"$15,001 - $50,000",Waste Connections/Progressive Waste stock merg...,Susan M Collins,2016,6,1,15001,50000
5021,2016-01-19,Spouse,CB,Exchange,"$1,001 - $15,000",MANDATORY MERGER FROM CHUBB CORPORATION,"Angus S King, Jr.",2016,1,19,1001,15000
5262,2015-07-06,Self,KHC,Exchange,"$250,001 - $500,000",Recived cash and shares of KHC in exchange for...,John Hoeven,2015,7,6,250001,500000
5263,2015-07-06,Self,KRFT,Exchange,"$250,001 - $500,000","Merger of KRFT and KHC, results in receiving c...",John Hoeven,2015,7,6,250001,500000


In [42]:
trades = []
for senator in senator_trades["senator"].unique():
    data = senator_trades[senator_trades["senator"] == senator]
    data = data.sort_values("transaction_date", ascending=True)

    temp = {}

    for row in data.itertuples():
        if row.type == "Sale (Full)": # or row["type"] == "Sale (Partial)":
            if row.ticker in temp:
                temp[row.ticker]["sell_date"] = row.transaction_date
                trades.append(temp[row.ticker])
                temp.pop(row.ticker)
        elif row.type == "Purchase":
            row_dict = {}
            for field in senator_trades.columns:
                row_dict[field] = getattr(row, field)
            temp[row.ticker] = row_dict


pd.DataFrame(trades)

,transaction_date,owner,ticker,type,amount,comment,senator,transaction_year,transaction_month,transaction_day,amount_lower,amount_upper,sell_date
0,2021-03-16,Joint,AA,Purchase,"$1,001 - $15,000",--,Thomas H Tuberville,2021,3,16,1001,15000,2021-04-19
1,2021-04-19,Joint,CLF,Purchase,"$50,001 - $100,000",--,Thomas H Tuberville,2021,4,19,50001,100000,2021-07-19
2,2015-08-25,Spouse,AEE,Purchase,"$1,001 - $15,000",--,Thomas R Carper,2015,8,25,1001,15000,2016-06-24
3,2017-06-28,Spouse,QCOM,Purchase,"$1,001 - $15,000",--,Thomas R Carper,2017,6,28,1001,15000,2019-04-17
4,2019-06-26,Spouse,BSTZ,Purchase,"$1,001 - $15,000",--,Thomas R Carper,2019,6,26,1001,15000,2020-02-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,2017-03-16,Spouse,DNKN,Purchase,"$1,001 - $15,000",--,Patty Murray,2017,3,16,1001,15000,2017-06-15
657,2017-03-16,Spouse,DUK,Purchase,"$1,001 - $15,000",--,Patty Murray,2017,3,16,1001,15000,2017-06-15
658,2017-03-16,Spouse,CB,Purchase,"$1,001 - $15,000",--,Patty Murray,2017,3,16,1001,15000,2017-06-15
659,2017-03-16,Spouse,MDT,Purchase,"$1,001 - $15,000",--,Patty Murray,2017,3,16,1001,15000,2017-06-15
